In [ ]:
# Check GPU
!nvidia-smi

# Mount Google Drive for saving checkpoints
from google.colab import drive
drive.mount('/content/drive')

Sat Dec  6 23:10:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
%cd /content
!rm -rf /content/Transferring_Time
!ls -al /content

/content
total 20
drwxr-xr-x 1 root root 4096 Dec  7 06:24 .
drwxr-xr-x 1 root root 4096 Dec  6 22:58 ..
drwxr-xr-x 4 root root 4096 Nov 20 14:30 .config
drwx------ 5 root root 4096 Dec  6 23:10 drive
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data


In [ ]:
!git clone https://github.com/JasonDai1219/Transferring_Time.git
%cd Transferring_Time

print("当前分支:")
!git branch

Cloning into 'Transferring_Time'...
remote: Enumerating objects: 284, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 284 (delta 109), reused 126 (delta 57), pack-reused 101 (from 1)
Receiving objects: 100% (284/284), 49.58 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (131/131), done.
/content/Transferring_Time
📂 当前分支:
* main


# Introduction

This is the training pipeline of the baseline CNN, the UNet, and the UNet + updated loss & weight map. Please run cell by cell to reproduce the output.

# Set up and Imports

In [ ]:
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy

from UNet_updated_loss_weight.src.train import main as train_main
plt.rcParams["figure.dpi"] = 120
plt.rcParams["figure.figsize"] = (6, 4)

print("CUDA available:", torch.cuda.is_available())

CUDA available: True


# Load Baseline Config

In [ ]:
def load_config(model_name):
    path = f"UNet_updated_loss_weight/configs/config_{model_name}.yaml"
    with open(path, "r") as f:
        return yaml.safe_load(f)

In [ ]:
cfg1 = load_config("cnn")          # simple_cnn baseline
cfg2 = load_config("unet")         # vanilla UNet
cfg3 = load_config("unet_advanced")# advanced UNet

In [ ]:
cfg1

{'dataset': {'image_size': 256, 'batch_size': 8, 'num_workers': 2},
 'training': {'model_type': 'simple_cnn',
  'epochs': 60,
  'lr': 0.0001,
  'loss_mode': 'baseline',
  'alpha': 0.0,
  'beta': 0.0,
  'lambda_edge': 0.0,
  'lambda_reflect': 0.0,
  'sigma_edge': 0.0},
 'logging': {'use_tensorboard': True,
  'log_dir': 'runs/cnn_baseline',
  'checkpoint_dir': 'checkpoints/cnn_baseline'}}

# Helper Function definition

Make experiemnt function caller cleaner

In [ ]:
import subprocess
import json

def run_experiment(config_path):
    print(f"Running experiment with config: {config_path}")

    process = subprocess.Popen(
        ["python3", "-m", "UNet_updated_loss_weight.src.train", "--config", config_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    logs = []
    for line in process.stdout:
        print(line, end="")
        logs.append(line)

    process.wait()
    return logs

# Baseline CNN

In [ ]:
cnn_cfg = "UNet_updated_loss_weight/configs/config_cnn.yaml"
cnn_logs = run_experiment(cnn_cfg)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 12: 100%|██████████| 499/499 [01:17<00:00,  6.47it/s]
Epoch 12 avg train loss: 2.0772
Epoch 12 avg val loss: 2.1152 | mIoU: 0.0567

Epoch 13: 100%|██████████| 499/499 [01:18<00:00,  6.35it/s]
Epoch 13 avg train loss: 2.0698
Epoch 13 avg val loss: 2.0928 | mIoU: 0.0565

Epoch 14: 100%|██████████| 499/499 [01:19<00:00,  6.29it/s]
Epoch 14 avg train loss: 2.0617
Epoch 14 avg val loss: 2.0862 | mIoU: 0.0573

Epoch 15: 100%|██████████| 499/499 [01:17<00:00,  6.40it/s]
Epoch 15 avg train loss: 2.0541
Epoch 15 avg val loss: 2.0693 | mIoU: 0.0657
🌟 Best model updated (mIoU = 0.0657)

Epoch 16: 100%|██████████| 499/499 [01:19<00:00,  6.30it/s]
Epoch 16 avg train loss: 2.0492
Epoch 16 avg val loss: 2.0683 | mIoU: 0.0597

Epoch 17: 100%|██████████| 499/499 [01:19<00:00,  6.29it/s]
Epoch 17 avg train loss: 2.0410
Epoch 17 avg val loss: 2.0563 | mIoU: 0.0614

Epoch 18: 100%|██████████| 499/499 [01:18<00:00,  6.37it/s]
Epoch 18 avg train loss: 2.0342
Epoch 18 avg val

# UNet

In [ ]:
cfg2

{'dataset': {'image_size': 256, 'batch_size': 8, 'num_workers': 2},
 'training': {'model_type': 'unet',
  'epochs': 60,
  'lr': 0.0001,
  'loss_mode': 'vanilla',
  'alpha': 0.5,
  'beta': 0.0,
  'lambda_edge': 0.0,
  'lambda_reflect': 0.0,
  'sigma_edge': 0.0},
 'logging': {'use_tensorboard': True,
  'log_dir': 'runs/unet',
  'checkpoint_dir': 'checkpoints/unet'}}

In [ ]:
unet_cfg = "UNet_updated_loss_weight/configs/config_unet.yaml"
unet_logs = run_experiment(unet_cfg)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 19: 100%|██████████| 499/499 [01:16<00:00,  6.49it/s]
Epoch 19 avg train loss: 1.7323
Epoch 19 avg val loss: 1.8740 | mIoU: 0.1216

Epoch 20: 100%|██████████| 499/499 [01:17<00:00,  6.44it/s]
Epoch 20 avg train loss: 1.6933
Epoch 20 avg val loss: 1.8691 | mIoU: 0.1276
🌟 Best model updated (mIoU = 0.1276)

Epoch 21: 100%|██████████| 499/499 [01:18<00:00,  6.33it/s]
Epoch 21 avg train loss: 1.6534
Epoch 21 avg val loss: 1.8853 | mIoU: 0.1216

Epoch 22: 100%|██████████| 499/499 [01:20<00:00,  6.18it/s]
Epoch 22 avg train loss: 1.6139
Epoch 22 avg val loss: 1.8607 | mIoU: 0.1202

Epoch 23: 100%|██████████| 499/499 [01:18<00:00,  6.37it/s]
Epoch 23 avg train loss: 1.5818
Epoch 23 avg val loss: 1.8420 | mIoU: 0.1213

Epoch 24: 100%|██████████| 499/499 [01:19<00:00,  6.27it/s]
Epoch 24 avg train loss: 1.5393
Epoch 24 avg val loss: 1.8391 | mIoU: 0.1198

Epoch 25: 100%|██████████| 499/499 [01:18<00:00,  6.34it/s]
Epoch 25 avg train loss: 1.5022
Epoch 25 avg val

# UNet + Updated Loss & Weight Map

## Define Search Space

In [ ]:
search_space = {
    "lambda_edge":    [0.3, 0.7, 1.2],
    "lambda_reflect": [0.02, 0.05],
    "alpha":          [0.5],
    "beta":           [0.15, 0.25, 0.35],
}

In [ ]:
def quick_eval(config_path, epochs=5, batch_size=4):
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)

    cfg["training"]["epochs"] = epochs
    cfg["dataset"]["batch_size"] = batch_size

    temp_cfg = "__temp_quick_eval.yaml"
    with open(temp_cfg, "w") as f:
        yaml.dump(cfg, f)

    logs = run_experiment(temp_cfg)

    for line in reversed(logs):
        if "| mIoU:" in line:
            try:
                return float(line.strip().split("mIoU:")[-1])
            except:
                pass

    return 0.0

In [ ]:
import itertools

def grid_search(base_cfg_path, search_space):
    param_keys = list(search_space.keys())
    combinations = list(itertools.product(*search_space.values()))
    total = len(combinations)

    results = []

    print(f"\n🔥 Starting Grid Search… {total} combinations total.\n")

    for idx, values in enumerate(combinations, start=1):
        params = dict(zip(param_keys, values))

        print(f"\n🔢 Progress: {idx} / {total}")
        print(f"🔎 Testing params: {params}")

        # Load base config
        with open(base_cfg_path, "r") as f:
            cfg = yaml.safe_load(f)

        # Inject params
        for k, v in params.items():
            cfg["training"][k] = v

        # Save temporary config
        temp_cfg = "__temp_search.yaml"
        with open(temp_cfg, "w") as f:
            yaml.dump(cfg, f)

        # Quick evaluation
        miou = quick_eval(temp_cfg)

        print(f"📊 mIoU = {miou:.4f}")

        results.append((miou, params))

    # Sort by score (descending)
    results.sort(reverse=True, key=lambda x: x[0])

    print("\n🏁 Grid Search Completed! Top results:")
    for score, param in results[:5]:
        print(f"⭐ mIoU {score:.4f} → {param}")

    return results

In [ ]:
base_cfg = "UNet_updated_loss_weight/configs/config_unet_advanced.yaml"
results = grid_search(base_cfg, search_space)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 2: 100%|██████████| 997/997 [01:27<00:00, 11.45it/s]
Epoch 2 avg train loss: 2.6055
Epoch 2 avg val loss: 2.5223 | mIoU: 0.0816

Epoch 3: 100%|██████████| 997/997 [01:25<00:00, 11.68it/s]
Epoch 3 avg train loss: 2.5055
Epoch 3 avg val loss: 2.4439 | mIoU: 0.0890
🌟 Best model updated (mIoU = 0.0890)

Epoch 4: 100%|██████████| 997/997 [01:27<00:00, 11.38it/s]
Epoch 4 avg train loss: 2.4305
Epoch 4 avg val loss: 2.4169 | mIoU: 0.0867
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.64].
✅ Final Test Loss: 2.412799 | Test mIoU: 0.0896
Figure(1200x400)
📊 mIoU = 0.0867

🔢 Progress: 17 / 18
🔎 Testing params: {'lambda_edge': 1.2, 'lambda_reflect': 0.05, 'alpha': 0.5, 'beta': 0.25}
🚀 Running experiment with config: __temp_quick_eval.yaml
2025-12-07 06:00:22.446950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to regi

# Unet + Updated Loss & Weight Map

In [ ]:
cfg3

{'dataset': {'image_size': 256, 'batch_size': 8, 'num_workers': 2},
 'training': {'model_type': 'unet',
  'epochs': 60,
  'lr': 0.0001,
  'loss_mode': 'advanced',
  'alpha': 0.5,
  'beta': 0.25,
  'lambda_edge': 0.7,
  'lambda_reflect': 0.02,
  'sigma_edge': 3.0},
 'logging': {'use_tensorboard': True,
  'log_dir': 'runs/unet_final',
  'checkpoint_dir': 'checkpoints/unet_final'}}

In [ ]:
unet_updated_cfg = "UNet_updated_loss_weight/configs/config_unet_advanced.yaml"
unet_updated_logs = run_experiment(unet_updated_cfg)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 12: 100%|██████████| 499/499 [01:18<00:00,  6.38it/s]
Epoch 12 avg train loss: 2.1198
Epoch 12 avg val loss: 2.1747 | mIoU: 0.1094

Epoch 13: 100%|██████████| 499/499 [01:17<00:00,  6.45it/s]
Epoch 13 avg train loss: 2.0814
Epoch 13 avg val loss: 2.1534 | mIoU: 0.1140
🌟 Best model updated (mIoU = 0.1140)

Epoch 14: 100%|██████████| 499/499 [01:18<00:00,  6.38it/s]
Epoch 14 avg train loss: 2.0469
Epoch 14 avg val loss: 2.0846 | mIoU: 0.1143
🌟 Best model updated (mIoU = 0.1143)

Epoch 15: 100%|██████████| 499/499 [01:20<00:00,  6.20it/s]
Epoch 15 avg train loss: 2.0103
Epoch 15 avg val loss: 2.0775 | mIoU: 0.1155
🌟 Best model updated (mIoU = 0.1155)

Epoch 16: 100%|██████████| 499/499 [01:17<00:00,  6.46it/s]
Epoch 16 avg train loss: 1.9727
Epoch 16 avg val loss: 2.0534 | mIoU: 0.1192
🌟 Best model updated (mIoU = 0.1192)

Epoch 17: 100%|██████████| 499/499 [01:17<00:00,  6.41it/s]
Epoch 17 avg train loss: 1.9342
Epoch 17 avg val loss: 2.0533 | mIoU: 0.113